# Space Invaders - un «envahisseur»

[Vidéo d'accompagnement](https://vimeo.com/502720878)

Notre «hero» et les «missiles» ont pas mal de code similaire et nous allons ajouter une nouvelle classe pour représenter les «envahisseurs».

Cela risque de produire beaucoup de **duplication de code** ce qui n'est **pas bon du tout**!!

En effet, si on modifie le code d'un fichier, il va falloir faire la même modification dans tous les autres!

## L'héritage pour «factoriser» le code

Voyons cela sur un exemple:

In [ ]:
class A:
    def __init__(self, a):
        self.a = a

    def fn1(self):
        print("faire qqchose")

    def fn2(self, x):
        print(self.a + x)


# test
a1, a2 = A(10), A(100)
a1.fn1()
a1.fn2(5)  # 15
a2.fn2(5)  # 105

 Voici maintenant un exemple d'**héritage**:

In [ ]:
class B(A):
    def __init__(self, val):
        super().__init__(val)

    def fn2(self, x):
        print(self.a * x)
        super().fn2(x)


# test
b = B(10)
print(b.a)  # 5
b.fn1()
b.fn2(5)  # 50 puis 15

Observer bien:
- l'attribut `a` n'est défini nul part dans la classe `B` et il est pourtant accessible depuis cette classe... car il a été *hérité* de la classe `A`,
- de même `fn1` n'est pas définie dans `B` mais elle est héritée de `A`,
- par contre, `fn2` est re-définie dans `B`, noter qu'il est tout de même encore possible d'accéder à la fonction de même nom définie dans `A`.


### Résumé de la syntaxe

- Pour qu'une classe `B` («fille») **hérite** d'une classe `A` («mère»), on écrit:
    - `class B(A)`.
- Pour pour utiliser une fonction `fn` définie dans la classe «mère» `A` **depuis** la classe «fille» `B`, on écrit:
    - `self.fn()` si la fonction n'est **pas redéfinie**,
    - `super().fn()` sinon.

*NOTES*: 
- la syntaxe `super().fn` est souvent la première instruction d'une fonction `fn` *re-définie*; c'est obligatoire pour la fonction spéciale `__init__`.
- Dans `B`, il n'était pas nécessaire de redéfinir `__init__` car elle ne fait rien d'autre que d'appeler l'initialisateur de la classe mère ce qui est le comportement par défaut (vous pouvez la supprimer, ça ne change rien.)

## Interface de la classe mère `Acteur`

Nous souhaitons réaliser une classe `Envahisseur`, mais elle risque d'avoir de nombreux points communs avec `Hero` et `Missile`. 

Nous allons donc créer une classe `Acteur` qui va servir de classe «mère» à nos trois classes, voici son «interface»:

```python
# module space_invaders/acteur.py


class Acteur:
    def __init__(self, scene, largeur, hauteur, couleur="white"):
        # pour accéder facilement aux informations de contexte
        self.scene = scene
        self.fen = scene.winfo_toplevel()
        self.L = int(scene["width"])
        self.H = int(scene["height"])

        # attributs propres de chaque acteur
        self.dim = largeur, hauteur
        self.pos = 0, 0
        self.v = 0  # vitesse en pixel par seconde
        self.id = scene.create_rectangle(0, 0, largeur, hauteur, fill=couleur)

    def supprimer(self):
        "Efface l'acteur de la scene"
        pass

    def set_position(self, x, y):
        "Met l'acteur dans la position indiquée"
        pass

    def deplacer(self, dx, dy):
        "Déplace l'acteur de dx pixels horizontalement et de dy pixels verticalement"
        pass

    def deplacer_duree(self, duree, direction="h"):
        """Déplace l'acteur en utilisant sa vitesse (horizontalement par défaut).
        l'unité de durée est la seconde"""
        pass

    def set_vitesse(self, v):
        pass

    def reagir(self, evt_type, gestionnaire):
        pass

    def est_dans_scene(self, dx=0, dy=0):
        """Test si l'acteur est entièrement dans la scène,
        après un déplacement (virtulel) dx, dy s'il est précisé"""
        pass

    def est_hors_scene(self):
        """Test si l'acteur est entièrement hors de la scène"""
        pass


if __name__ == "__main__":
    # zone pour tester son code
    pass
```

L'étape qui consiste à **bien réfléchir à l'interface** d'une classe, c'est-à-dire:

au choix du *rôle* et du *nom* de chaque attribut et fonction, et des *paramètres* et type de la valeur retournée s'il y en a.

est **CRUCIALE**.

En effet, c'est par l'intermédiaire de cette interface qu'on va manipuler chaque acteur: hero, missiles, envahisseurs.

**Le piège du débutant**: *vouloir écrire trop rapidement le code de chaque fonction* avant d'avoir bien fixé l'*interface*.

## Jeu: réécrire la classe Missile avant d'avoir fini `Acteur`

Essais de compléter le code manquant en utilisant l'interface de la classe acteur:

```python
# module space_invaders/missile.py
from acteur import Acteur


class Missile(___):
    def __init__(self, scene, couleur="white"):
        super().__init__(scene, largeur=__, hauteur=__, couleur=__)

    def lancer(self, duree_milli=50):
        # si le missile est sorti de la scène (entièrement) le détruire
        if self.___:
            self.___
            return

        # à présent, on peut agir ...
        self.deplacer_duree(___)

        # et on recommence
        self.scene.after(duree_milli, lambda: self.lancer(duree_milli))

# ...
```

**Solution**

```python
# module space_invaders/missile.py
from acteur import Acteur


class Missile(Acteur):
    def __init__(self, scene, couleur="white"):
        super().__init__(scene, largeur=5, hauteur=15, couleur=couleur)

    def lancer(self, duree_milli=50):
        # si le missile est sorti de la scène (entièrement) le détruire
        if self.est_hors_scene():
            self.supprimer()
            return

        # à présent, on peut agir ...
        self.deplacer_duree(duree_milli / 1000)

        # et on recommence
        self.scene.after(duree_milli, lambda: self.lancer(duree_milli))
```

## Implémentation de `Acteur`

Complète le code des fonctions indiquées.

### `supprimer`

```python
def supprimer(self):
    "Efface l'acteur de la scene"

    ___.delete(self.__)
```

**Solution**

```python
def supprimer(self):
    "Efface l'acteur de la scene"

    self.scene.delete(self.id)
```

### `set_position`

```python
def set_position(self, x, y):
    "Met l'acteur dans la position indiquée"

    x0, y0 = ___
    self.pos = x, y
    self.scene.___(self.___, x - x0, y - y0)
```

**Solution**

```python
def set_position(self, x, y):
    "Met l'acteur dans la position indiquée"

    x0, y0 = self.pos
    self.pos = x, y
    self.scene.move(self.id, x - x0, y - y0)
```

### `deplacer` et `deplacer_duree`

```python
def deplacer(self, dx, dy):
    "Déplace l'acteur de dx pixels horizontalement et de dy pixels verticalement"

    x0, y0 = self.pos
    ___(x0 + __, y0 + __)


def deplacer_duree(self, duree, direction="h"):
    """Déplace l'acteur en utilisant sa vitesse (horizontalement par défaut)
        «duree» est supposé être en seconde
        """

    if direction == "h":
        dx = self.v * ___
        self.__(dx, 0)
    else:
        ___
```

**Solution**

```python
def deplacer(self, dx, dy):
    "Déplace l'acteur de dx pixels horizontalement et de dy pixels verticalement"

    x0, y0 = self.pos
    self.set_position(x0 + dx, y0 + dy)


def deplacer_duree(self, duree, direction="h"):
    """Déplace l'acteur en utilisant sa vitesse (horizontalement par défaut)
        «duree» est supposé être en seconde
        """

    if direction == "h":
        dx = self.v * duree
        self.deplacer(dx, 0)
    else:
        dy = self.v * duree
        self.deplacer(0, dy)
```

### `est_dans_scene` et `est_hors_scene`

```python
def est_hors_scene(self):
    """Test si l'acteur est entièrement hors de la scène"""

    x, y = ___
    l, h = ___
    return x + l < __ or x >= __ or ___ < 0 or y >= ___


def est_dans_scene(self, dx=0, dy=0):
    """Test si l'acteur est entièrement dans la scène,
        après un déplacement (virtulel) dx, dy s'il est précisé"""

    x, y = ___
    x, y = x + ___, y + ___
    l, h = ___
    return ___
```

**Solution**

```python
def est_hors_scene(self):
    """Test si l'acteur est entièrement hors de la scène"""

    x, y = self.pos
    l, h = self.dim
    return x + l < 0 or x >= self.L or y + h < 0 or y >= self.H


def est_dans_scene(self, dx=0, dy=0):
    """Test si l'acteur est entièrement dans la scène,
        après un déplacement (virtulel) dx, dy s'il est précisé"""

    x, y = self.pos
    x, y = x + dx, y + dy
    l, h = self.dim
    return x >= 0 and x + l <= self.L and y >= 0 and y + h <= self.H
```

### `set_vitesse` et `reagir`

voici la solution pour abréger!

```python
def set_vitesse(self, v):
    self.v = v


def reagir(self, evt_type, gestionnaire):
    return self.fen.bind(evt_type, lambda _: gestionnaire())
```

### Penser à tester

```python
class Acteur:
    #...

if __name__ == "__main__":
    # zone pour tester son code
    from scene import *

    acteur = Acteur(scene, 100, 100)
    acteur.set_position(acteur.L - 50, 100)
    assert not acteur.est_dans_scene()
    assert acteur.est_hors_scene()
    scene.pack()
    acteur.reagir("<Left>", lambda: acteur.deplacer(-5, 0))
    acteur.reagir("<Right>", lambda: acteur.deplacer(5, 0))
    fen.mainloop()
```

Cela produit une erreur, voyez-vous pourquoi?

La deuxième assertion est fausse: l'acteur n'est ni entièrement dans la scène, ni entièrement en dehors (il est à cheval...).

`assert not acteur.est_hors_scene()` pour supprimer l'erreur.

## Réécriture de `Hero` 

Complète les parties manquantes.

```python
from acteur import Acteur
from missile import Missile

class Hero(Acteur):
    
    def __init__(self, scene, couleur="white"):
        super().__init__(scene, largeur=__, hauteur=__, couleur=__)
        
        self.reagir('<Left>', self.__)
        self.reagir('<Right>', self.__)
        self.reagir('<space>', self.__)
        
        self.PAS = 10
        self.set_position(self.L//2, self.H-50)
    
    def gauche(self):
        if self.est_dans_scene(__, __):
            self.deplacer(-self.PAS, 0)
    
    def droite(self):
        # à toi de jouer
        pass
    
    def tirer(self):
        x, y = self.pos
        l, _ = self.dim
        m = Missile(___)
        _, hm = m.dim
        m.____(x + l//2, y - hm)
        m.____(-300)
        m.lancer()
    

if __name__ == "__main__":
    from scene import *
    hero = Hero(scene)
    fen.mainloop()
```

**Solution**

```python
from acteur import Acteur
from missile import Missile

class Hero(Acteur):
    
    def __init__(self, scene, couleur="white"):
        super().__init__(scene, largeur=50, hauteur=30, couleur=couleur)
        
        self.reagir('<Left>', self.gauche)
        self.reagir('<Right>', self.droite)
        self.reagir('<space>', self.tirer)
        
        self.PAS = 10
        self.set_position(self.L//2, self.H-50)
    
    def gauche(self):
        if self.est_dans_scene(-self.PAS, 0):
            self.deplacer(-self.PAS, 0)
    
    def droite(self):
        if self.est_dans_scene(self.PAS, 0):
            self.deplacer(self.PAS, 0)
    
    def tirer(self):
        x, y = self.pos
        l, _ = self.dim
        m = Missile(self.scene)
        _, hm = m.dim
        m.set_position(x + l//2, y - hm)
        m.set_vitesse(-300)
        m.lancer()
    

if __name__ == "__main__":
    from scene import *
    hero = Hero(scene)
    fen.mainloop()
```

## Écriture de `Envahisseur`

Nous souhaitons que l'envahisseur (rouge par défaut):
- parcourt le haut de l'écran de gauche à droite ou le contraire
- tout en lançant des missiles au hasard.

Il ressemble donc à un héro (lance des missiles) et à un missile (est lancé lui-même).

Réalise ce comportement en complétant le code qui suit.

```python
# dans space-invaders/envahisseur.py
from acteur import Acteur
from missile import Missile
from random import choice, random


class Envahisseur(Acteur):
    def __init__(self, scene, couleur="red"):
        super().__init__(scene, largeur=50, hauteur=30, couleur=__)
        
        # choisir une vitesse au hasard
        self.v = choice([-100, 100])

        # régler la position de façon à apparaîte en haut
        # à gauche ou à droite selon la vitesse ...
        l, _ = self.dim
        if self.v > 0:
            self.___(-l + 10, 30)
        else:
            self.set_position(___)

        # tout est prêt! lançons le.
        self._lancer()

    # un underscore devant une fonction est une *convention* pour
    # indiquer que celle-ci ne doit pas être utilisée en dehors de la classe
    def _lancer(self, duree_milli=50):
        # revoie le code similaire de Missile ...
        ________

        # random() renvoie un nombre dans [0;1]
        # utilisons le pour tirer aléatoirement
        if random() < 0.025:
            ____

        ________

    def tirer(self):
        # revoie le code similaire de Hero ...
        _____

if __name__ == "__main__":
    from scene import *
    from hero import Hero

    env = Envahisseur(scene)
    hero = Hero(scene)
    fen.mainloop()
```

**Solution**

```python
from acteur import Acteur
from missile import Missile
from random import choice, random


class Envahisseur(Acteur):
    def __init__(self, scene, couleur="red"):
        super().__init__(scene, largeur=50, hauteur=30, couleur=couleur)
        # choisir une vitesse au hasard
        self.v = choice([-100, 100])

        # régler la position de façon à apparaîte en haut
        # à gauche ou à droite selon la vitesse ...
        l, _ = self.dim
        if self.v > 0:
            self.set_position(-l + 10, 30)
        else:
            self.set_position(self.L - 10, 30)

        # tout est prêt! lançons le.
        self._lancer()

    # un underscore devant une fonction est une *convention* pour
    # indiquer que celle-ci ne doit pas être utilisée en dehors de la classe
    def _lancer(self, duree_milli=50):
        if self.est_hors_scene():
            self.supprimer()
            return

        self.deplacer_duree(duree_milli / 1000)

        # random() renvoie un nombre dans [0;1]
        # utilisons le pour tirer aléatoirement
        if random() < 0.025:
            self.tirer()

        self.scene.after(
            duree_milli, lambda: self._lancer(duree_milli),
        )

    def tirer(self):
        x, y = self.pos
        l, h = self.dim
        m = Missile(self.scene)
        m.set_position(x + l // 2, y + h)
        m.set_vitesse(300)
        m.lancer()

    def __del__(self):
        print("supprimer")


if __name__ == "__main__":
    from scene import *
    from hero import Hero

    env = Envahisseur(scene)
    hero = Hero(scene)
    fen.mainloop()
```